# COLAB SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys, os

if 'google.colab' in sys.modules:
    ! git clone https://github.com/birshert/attack-lang-models

    %cd attack-lang-models
    
    ! pip install -r requirements.txt
    
    import nltk
    nltk.download('punkt')
    
    ! cp /content/drive/MyDrive/*.pth /content/attack-lang-models/models/

# MAIN BODY

In [1]:
import model

In [2]:
model_ = model.Model()

In [3]:
model_.russian_forward()

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [5]:
import torch

a = tokenizer.encode("My cat is big.", return_tensors='pt')
b = tokenizer.encode("My <mask> is big.", return_tensors='pt')

In [6]:
optimizer = torch.optim.Adam(model_.mlm_model.parameters())

In [7]:
for i in range(30):
    optimizer.zero_grad()
    loss = model_(a, b)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        logits = model_.mlm_model(b).logits
        token = torch.argmax(logits[:, torch.where(b == tokenizer.mask_token_id)[1], :]).item()

        print(f'token {tokenizer.decode([token])}, loss {loss.item():.5f}')

token словом, loss -0.65785
token сам, loss -0.67669
token перспектива, loss -0.66267
token перспектива, loss -0.65391
token перспектива, loss -0.65921
token перспектива, loss -0.65391
token перспектива, loss -0.65391
token брата, loss -0.54360
token брата, loss -0.54360
token брата, loss -0.65921
token брата, loss -0.65921
token брата, loss -0.54360
token брата, loss -0.54360
token брата, loss -0.65391
token брата, loss -0.54360
token брата, loss -0.54360
token брата, loss -0.54360
token брата, loss -0.54360
token брата, loss -0.54360
token брата, loss -0.68976
token брата, loss -0.54360


KeyboardInterrupt: 

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')

In [23]:
a = tokenizer.encode("My cat is big.", return_tensors='pt')
b = tokenizer.encode("My животное is big.", return_tensors='pt')

t1 = model_.emb_model(a).pooler_output
t2 = model_.emb_model(b).pooler_output

normalized_embeddings_1 = torch.nn.functional.normalize(t1, p=2)
normalized_embeddings_2 = torch.nn.functional.normalize(t2, p=2)

torch.sum(normalized_embeddings_1 * normalized_embeddings_2, dim=1).mean()

tensor(0.6534)